<a href="https://colab.research.google.com/github/MaxKuzaj13/dw_matrix_road_sign/blob/master/matrix_3_day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'/content'

In [3]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import datetime
import os


#%load_ext tensorboard
import matplotlib.pyplot as plt
from skimage import color,exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [6]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
df = pd.read_csv('data/signnames.csv')
labels_dict =df.to_dict()['b']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)


In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_cnn_v1(input_shape, num_classes):
  return  Sequential([
                  Conv2D(filters=65, kernel_size=(3,3), activation='relu',input_shape=input_shape),
                  Flatten(),
                  Dense(num_classes, activation='softmax'),
])

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss = 'categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])

  logdir=os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  return model 

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return  Sequential([
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(0.3),

                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                  MaxPool2D(),
                  Dropout(0.3),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                  MaxPool2D(),
                  Dropout(0.3),

                  Flatten(),
                  Dense(1024, activation= 'relu'),
                  Dropout(0.3),
                  Dense(1024, activation= 'relu'),
                  Dropout(0.3),

                  Dense(num_classes, activation='softmax'),
])

In [17]:
model  = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.5039 - accuracy: 0.2891 - val_loss: 1.4053 - val_accuracy: 0.5249
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.0517 - accuracy: 0.6554 - val_loss: 0.2446 - val_accuracy: 0.9230
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3849 - accuracy: 0.8787 - val_loss: 0.0939 - val_accuracy: 0.9725
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2318 - accuracy: 0.9290 - val_loss: 0.0653 - val_accuracy: 0.9810
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1833 - accuracy: 0.9459 - val_loss: 0.0323 - val_accuracy: 0.9912


0.9659863945578231

In [19]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1461 - accuracy: 0.9660


0.9659863710403442

In [0]:
def get_model(params):
  return  Sequential([
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(params['dropout_cnn_block_one']),

                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

                  MaxPool2D(),
                  Dropout(params['dropout_cnn_block_two']),
                  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),

                  MaxPool2D(),
                  Dropout(params['dropout_cnn_block_three']),

                  Flatten(),
                  Dense(1024, activation= 'relu'),
                  Dropout(params['dropout_dense_block_one']),
                  Dense(1024, activation= 'relu'),
                  Dropout(params['dropout_dense_block_two']),

                  Dense(num_classes, activation='softmax'),
])

In [0]:
def func_obj(params):
  model =  get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])


  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
  )

  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status':STATUS_OK, 'model': model}

In [35]:
space = {
    'batch_size' : hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one' :  hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two' :  hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}
best = fmin(
    func_obj, 
    space, 
    tpe.suggest, 
    30, 
    Trials()
)

{'batch_size': 150.0, 'dropout_cnn_block_one': 0.38605352925930203, 'dropout_cnn_block_three': 0.4446325595273017, 'dropout_cnn_block_two': 0.4209915509556511, 'dropout_dense_block_one': 0.693364925857152, 'dropout_dense_block_two': 0.6663907549077142}
accuracy=0.7950113415718079
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4433491749951003, 'dropout_cnn_block_three': 0.4612848418508583, 'dropout_cnn_block_two': 0.47756306441664803, 'dropout_dense_block_one': 0.6625561184671376, 'dropout_dense_block_two': 0.5903412455801265}
accuracy=0.44195011258125305
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.38991298928728435, 'dropout_cnn_block_three': 0.4116095669083959, 'dropout_cnn_block_two': 0.44490414255188787, 'dropout_dense_block_one': 0.5840844292720236, 'dropout_dense_block_two': 0.568868629037986}
accuracy=0.855555534362793
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.49696444541645396, 'dropout_cnn_block_three': 0.3672309567632586, 'dropout_cnn_block_two': 0.3201188278397

In [0]:
##JESCZE NAQ FILMIE BYLO WIECEJ